In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import wandb

In [2]:
# Model
import torch.nn as nn
class SEBlock(nn.Module):
    """ Squeeze-and-Excitation Block """
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.se(x)
        return x * scale

class MultiHeadAttention(nn.Module):
    """ Multi-Head Attention Module """
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.attention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        return attn_output

class RadioNet(nn.Module):
    def __init__(self, num_classes):
        super(RadioNet, self).__init__()

        # Separate Convolutional Pathways for I and Q
        self.q_conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            SEBlock(64),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            SEBlock(256),
            nn.MaxPool2d(2, stride=2)
        )

        self.i_conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            SEBlock(64),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            SEBlock(256),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_size = self._get_conv_output((1, 32, 32))

        # Bidirectional LSTM with Layer Normalization
        self.lstm = nn.LSTM(self.feature_size * 2, 512, num_layers=2, 
                            batch_first=True, bidirectional=True, dropout=0.3)
        self.layer_norm = nn.LayerNorm(1024)  # Layer normalization after LSTM

        # Multi-Head Attention with multiple heads
        self.multi_head_attn = MultiHeadAttention(1024, num_heads=8)

        # Enhanced Fully Connected Layers with Dense Connections
        self.fc = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.LeakyReLU(0.1)
        )

        self.output = nn.Linear(64, num_classes)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        output = self.q_conv(input)
        return int(torch.numel(output) / output.shape[0])

    def forward(self, i_input, q_input):
        q = self.q_conv(q_input)
        q = q.view(q.size(0), -1)

        i = self.i_conv(i_input)
        i = i.view(i.size(0), -1)

        combined = torch.cat((q, i), dim=1)
        combined = combined.unsqueeze(1)  # Add sequence dimension

        lstm_out, _ = self.lstm(combined)
        lstm_out = self.layer_norm(lstm_out)

        # Apply Multi-Head Attention
        attn_output = self.multi_head_attn(lstm_out)
        context = torch.sum(attn_output, dim=1)  # Sum up the attended output

        x = self.fc(context)
        x = self.output(x)

        return torch.log_softmax(x, dim=1)

# def create_model(num_classes):
#     model = RadioNet(num_classes)
#     learning_rate = 0.0003
#     optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
#     loss_fn = nn.CrossEntropyLoss()
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
#     return model, optimizer, loss_fn, scheduler

In [3]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
trained_model = RadioNet(num_classes=8)

# Load the state dict, mapping it to the available device
trained_model.load_state_dict(torch.load('/kaggle/input/radionet/pytorch/default/1/model_checkpoint.pth', map_location=device))

# Move the model to the appropriate device
trained_model = trained_model.to(device)

trained_model.eval()

/tmp/ipykernel_17/3592900082.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_model.load_state_dict(torch.load('/kaggle/input/radionet/pytorch/default/1/model_che

RadioNet(
  (q_conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): SEBlock(
      (se): Sequential(
        (0): AdaptiveAvgPool2d(output_size=1)
        (1): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1))
        (2): ReLU()
        (3): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
        (4): Sigmoid()
      )
    )
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.1)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.1)

In [4]:
class FeatureExtractor(nn.Module):
    def __init__(self, trained_model):
        super(FeatureExtractor, self).__init__()
        self.q_conv = trained_model.q_conv
        self.i_conv = trained_model.i_conv
        self.lstm = trained_model.lstm
        self.layer_norm = trained_model.layer_norm
        self.multi_head_attn = trained_model.multi_head_attn
        self.fc = trained_model.fc

    def forward(self, i_input, q_input):
        q = self.q_conv(q_input)
        q = q.view(q.size(0), -1)

        i = self.i_conv(i_input)
        i = i.view(i.size(0), -1)

        combined = torch.cat((q, i), dim=1)
        combined = combined.unsqueeze(1)

        lstm_out, _ = self.lstm(combined)
        lstm_out = self.layer_norm(lstm_out)

        attn_output = self.multi_head_attn(lstm_out)
        context = torch.sum(attn_output, dim=1)

        features = self.fc(context)
        return features

feature_extractor = FeatureExtractor(trained_model)

In [5]:
def extract_features(model, i_input, q_input):
    with torch.no_grad():
        features = model(i_input, q_input)
    return features.cpu().numpy()

In [6]:
from torch.utils.data import Dataset, DataLoader
class RadioMLDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float().to(device)
        self.y = torch.from_numpy(y.values).float().to(device)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def extract_features(model, dataloader):
    features_list = []
    labels_list = []
    
    model.eval()
    with torch.no_grad():
        for batch_X, batch_y in dataloader:
            # Separate I and Q components
            i_input = batch_X[:, :, :, 0].unsqueeze(1)  # Shape: (batch_size, 1, 32, 32)
            q_input = batch_X[:, :, :, 1].unsqueeze(1)  # Shape: (batch_size, 1, 32, 32)
            
            features = model.fc(model.multi_head_attn(model.layer_norm(model.lstm(torch.cat((
                model.q_conv(q_input).view(q_input.size(0), -1),
                model.i_conv(i_input).view(i_input.size(0), -1)
            ), dim=1).unsqueeze(1))[0])).sum(dim=1))
            
            features_list.append(features.cpu().numpy())
            labels_list.append(batch_y.cpu().numpy())
    
    return np.vstack(features_list), np.vstack(labels_list)

In [7]:
# wandb login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key") 
wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
def xgboost_kfold_cv(X, y, n_splits=5, random_state=42):
    # Initialize wandb run
    wandb.init(project="radioml-xgboost", name="xgboost-kfold-cv")
    
    # Log config
    wandb.config.update({
        "n_splits": n_splits,
        "random_state": random_state,
        "model": "XGBoost"
    })

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_accuracies = []

    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        print(f"Fold {fold}")

        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=random_state)
        xgb_clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)

        y_pred = xgb_clf.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        fold_accuracies.append(accuracy)

        print(f"Fold {fold} Accuracy: {accuracy:.4f}")
        print(classification_report(y_val, y_pred))
        print("--------------------")

        # Log metrics for this fold
        wandb.log({
            f"fold_{fold}_accuracy": accuracy,
            "fold": fold
        })

        # Log feature importances
        feature_imp = xgb_clf.feature_importances_
        wandb.log({f"feature_importance_fold_{fold}": wandb.plot.bar(
            wandb.Table(data=[[f"feature_{i}", imp] for i, imp in enumerate(feature_imp)],
                        columns=["feature", "importance"]),
            "feature",
            "importance",
            title=f"Feature Importances (Fold {fold})"
        )})

    mean_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    print(f"Mean Accuracy: {mean_accuracy:.4f} (+/- {std_accuracy:.4f})")

    # Log final metrics
    wandb.log({
        "mean_accuracy": mean_accuracy,
        "std_accuracy": std_accuracy
    })

    # Finish the wandb run
    wandb.finish()

In [9]:
import h5py
# Read the dataset
dataset_file = h5py.File("/kaggle/input/radioml2018/GOLD_XYZ_OSC.0001_1024.hdf5", "r")

# Base modulation classes
base_modulation_classes = [
    'OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
    '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
    '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC',
    'FM', 'GMSK', 'OQPSK'
]

# Selected modulation classes
selected_modulation_classes = [
    '4ASK', 'BPSK', 'QPSK', '16PSK', '16QAM', 'FM', 'AM-DSB-WC', '32APSK'
]

# Get the indices of selected modulation classes
selected_classes_id = [base_modulation_classes.index(cls) for cls in selected_modulation_classes]

In [10]:
# Number of SNRs (from 30 SNR to 22 SNR)
N_SNR = 4 

# Initialize placeholders for data
X_data = None
y_data = None

# Loop through selected modulation classes
for id in selected_classes_id:
    # Load data slices based on indices
    X_slice = dataset_file['X'][(106496*(id+1) - 4096*N_SNR) : 106496*(id+1)]
    y_slice = dataset_file['Y'][(106496*(id+1) - 4096*N_SNR) : 106496*(id+1)]
    
    # Concatenate the slices to build the dataset
    if X_data is not None:
        X_data = np.concatenate([X_data, X_slice], axis=0)
        y_data = np.concatenate([y_data, y_slice], axis=0)
    else:
        X_data = X_slice
        y_data = y_slice

# Reshape the X_data to the required shape (e.g., 32x32 with 2 channels)
X_data = X_data.reshape(len(X_data), 32, 32, 2)

# Convert y_data to a DataFrame for easier manipulation
y_data_df = pd.DataFrame(y_data)

# Drop columns where the sum is 0 (i.e., no modulation class data in that column)
for column in y_data_df.columns:
    if sum(y_data_df[column]) == 0:
        y_data_df = y_data_df.drop(columns=[column])

# Assign the remaining columns to match the selected modulation classes
y_data_df.columns = selected_modulation_classes

# Split the dataset into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data_df, test_size=0.2)

In [11]:
# Create Dataset objects
train_dataset = RadioMLDataset(X_train, y_train)
test_dataset = RadioMLDataset(X_test, y_test)

# Create DataLoader objects
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Extract features
print("Extracting features from training data...")
X_train_features, y_train_features = extract_features(trained_model, train_loader)
print("Extracting features from test data...")
X_test_features, y_test_features = extract_features(trained_model, test_loader)

# Combine train and test features for K-fold CV
X_all_features = np.vstack((X_train_features, X_test_features))
y_all_features = np.vstack((y_train_features, y_test_features))

# Convert multi-hot encoded labels to class indices
y_all_indices = np.argmax(y_all_features, axis=1)

# Perform K-fold cross-validation with XGBoost
print("Performing K-fold cross-validation with XGBoost...")
xgboost_kfold_cv(X_all_features, y_all_indices)

Extracting features from training data...
Extracting features from test data...


wandb: Currently logged in as: devcode03 (devcode03-gujarat-technological-university). Use `wandb login --relogin` to force relogin


Performing K-fold cross-validation with XGBoost...


wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240912_113113-9nxvmuo2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run xgboost-kfold-cv
wandb: ⭐️ View project at https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost
wandb: 🚀 View run at https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost/runs/9nxvmuo2


Fold 1


...

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Fold 4 Accuracy: 0.9906
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3248
           1       1.00      1.00      1.00      3311
           2       0.99      0.99      0.99      3306
           3       1.00      1.00      1.00      3271
           4       0.96      0.97      0.97      3143
           5       0.97      0.96      0.97      3317
           6       1.00      1.00      1.00      3306
           7       1.00      1.00      1.00      3312

    accuracy                           0.99     26214
   macro avg       0.99      0.99      0.99     26214
weighted avg       0.99      0.99      0.99     26214

--------------------
Fold 5


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Fold 5 Accuracy: 0.9908
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3214
           1       1.00      1.00      1.00      3286
           2       1.00      1.00      1.00      3444
           3       1.00      1.00      1.00      3260
           4       0.97      0.98      0.97      3277
           5       0.97      0.96      0.96      3225
           6       1.00      1.00      1.00      3269
           7       1.00      1.00      1.00      3239

    accuracy                           0.99     26214
   macro avg       0.99      0.99      0.99     26214
weighted avg       0.99      0.99      0.99     26214

--------------------
Mean Accuracy: 0.9903 (+/- 0.0005)


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            fold ▁▃▅▆█
wandb: fold_1_accuracy ▁
wandb: fold_2_accuracy ▁
wandb: fold_3_accuracy ▁
wandb: fold_4_accuracy ▁
wandb: fold_5_accuracy ▁
wandb:   mean_accuracy ▁
wandb:    std_accuracy ▁
wandb: 
wandb: Run summary:
wandb:            fold 5
wandb: fold_1_accuracy 0.9905
wandb: fold_2_accuracy 0.98943
wandb: fold_3_accuracy 0.99001
wandb: fold_4_accuracy 0.99062
wandb: fold_5_accuracy 0.99081
wandb:   mean_accuracy 0.99027
wandb:    std_accuracy 0.0005
wandb: 
wandb: 🚀 View run xgboost-kfold-cv at: https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost/runs/9nxvmuo2
wandb: ⭐️ View project at: https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost
wandb: Synced 5 W&B file(s), 5 media file(s), 5 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240912_113113-9nxvmuo2/logs
wandb: WARNING The new W&